In [1]:
# this notebook will implement the predictive abrasion model using an explicit accounting of mass conservation

In [2]:
# first, outline the math

#### <i>CELL VOLUME</i>

$$ \text{volume} = whdx$$

where $w$ = width, $h$ = height, and $dx$ = length

<br>
<br>

#### <i>MASS</i>

$$m = \lambda \times \text{volume} \times (1 - \phi) \times \rho$$

where $\lambda$ = the percentage of a given material in the basin (igneous, metamorphic, or sedimentary), $\phi$ = porosity within the bed, and $\rho$ = the material density

<br>
<br>

#### <i>STATEMENT OF MASS CONSERVATION:</i>

$$ \text{rate of change of igneous mass} = \text{mass rate in} - \text{mass rate out} - \text{mass lost to abrasion}$$

A rate of mass change is $[M/t]$, so mass per time are the units we're shooting for everywhere.

<br>
<br>

#### <i>WRITING MASS CONSERVATION IN TERMS OF STUFF WE CAN CALCULATE IN OUR MODEL</i>

In our model, we can calculate a bedload sediment flux as a function of slope, discharge (itself a function of upstream contributing area), and some lumped factors that derive from equilibrium channel theory (excess shear stress, etc. See Wickert & Schildgen, 2019). This would look like

$$Q_s = c Q S$$

where $c$ is the lumped coefficient, $Q$ is discharge, and $S$ is bed slope. $c$ and $S$ are dimensionless, so $Q_s$ has the same dimensions as $Q$. Traditionally, dimensions of a water discharge or sediment yield would be $[V/t]$, volume per time, or $[L^3/t]$. Remember,

$$\text{mass} = \text{density} \times \text{volume}$$

$$[M] = [M/V] \times [V]$$

But, as a means of normalizing to channel width, we reduce the sediment flux to a unit sediment discharge, i.e. a flux per unit width. This can be written as

$$q_s = \frac{Q_s}{w}$$

Now $q_s$ represents the unit sediment flux, which has dimensions $[L^2/t]$ - basically, an area per time. This is a little bit of a problem since a volume per time is what we need to combine with material density in order to transform the dimesions of the right-hand side of our equation to the dimensions of the left-hand side, which are mass per time. If we write it all out, it looks like this:

$$\frac{\partial m}{\partial t} = \rho_{ig} q_s|_x w - \rho_{ig} q_s|_{x + dx} w - \rho_{ig} q_s|_x dx w \beta_{ig}$$

where:
- $\rho_{ig}$ is the density of igneous material, $[kg/m^{3}]$
- $q_s|_x$ is the unit sediment flux at the upstream end of a cell, $[m^2/s]$
- $q_s|_{x + dx}$ is the unit sediment flux at the downstream end of a cell, $[m^2/s]$
- $w$ is cell width, $[m]$ (this is used to transform the unit sediment flux back into a volumetric flux)
- $dx$ is cell length, $[m]$
- $\beta_{ig}$ is the abrasion coefficient, which is 1/sternberg length, or $[1/L]$

So altogether, this states that the mass rate of change is equal to the mass of sediment that enters a cell minus the mass of sediment that leaves a cell and minus the volume of material that is abraded across the surface area of the cell. We can rewrite the lefthand side as follows:

$$\frac{\partial m}{\partial t} = \frac{\partial \theta_{ig} \lambda whdx \rho_{ig}}{\partial t}$$

where:
- $\theta$ is the percentage of bed material of a given type (here igneous) $[\text{dimensionless}]$
- $\lambda$ is the percentage of the bed that is composed of solid material $(1 - \text{porosity})$ $[\text{dimensionless}]$
- $whdx$ is the cell volume (width times height time length) $[m^3]$
- $\rho_{ig}$ is the density (here of igneous material) $[kg/m^3]$

We can now pull all of the constants out of the partial derivative on the lefthand side and start canceling stuff. Without showing all the work, we should arrive at something like this:

$$\frac{\partial h}{\partial t} = -\frac{1}{\theta \lambda} \left (\frac{\partial q_s}{\partial x} + q_s\beta \right )$$

The steps we've taken that aren't shown here are:
- Take all constants ($\theta, \lambda, w, dx, \text{and} \rho$) out of the lefthand derivative and divide by these on the righthand side of the equation
- Cancel some terms
- Collapse the terms $\frac{q_s|_x}{dx}$ and $\frac{q_s|_{x+dx}}{dx}$ into a derivative (this includes changing the sign by dividing by $-1$) and rewrite as such

We'll now try to implement this in a model. We'll basically have to perform this calculation three times, once for each rock type we wish to represent (here, igneous, metamorphic, and sedimentary.)

In [3]:
# now, onto the modeling. Start by importing libraries.
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# define lithologic percentages
theta_ig = 0.70
theta_mtm = 0.05
theta_sed = 0.25

# now set up grid stuff
dx = 1
x = np.arange(0, 100, dx)
z = np.zeros(len(x))
qs = np.zeros(len(x))
baselevel_rate = 0.01
H = 2
h_ig = np.zeros(len(x))
h_ig[:] = H * theta_ig
h_mtm = np.zeros(len(x))
h_mtm[:] = H * theta_mtm
h_sed = np.zeros(len(x))
h_sed[:] = H * theta_sed

# define some constants
q = x
c = 1
porosity = 0.45

# define densities
# this isn't actually necessary
# rho_ig = 2650
# rho_mtm = 2650
# rho_sed = 2650

# define abrasion coefficients
beta_ig = 1
beta_mtm = 1e-5
beta_sed = 5e-5

# set runtime
num_steps = 1000
dt = (0.2 * dx * dx) / c

In [5]:
# now we want to write a loop to calculate elevation changes through time 
# and also track sediment flux of each of our different lithologies
# here are the steps:

# set downstream end of grid domain to lowered baselevel times dt
# calculate slope
# calculate sed flux from cqS


In [6]:
# writing and running the loop

for i in range(10000):
    
    # lower baselevel
    # z[-1] -= baselevel_rate * dt
    
    # calculate slope
    S = 0.001
    
    # discharge
    q = np.zeros(len(x))
    q[:] = 1
    
    # calculate sed flux
    qs[1:] = c * q[1:] * S
    qs[0] = 0
    qs_ig[1:] = qs[1:] * (h_ig[:-1]/H)
    qs_ig[0] = 0
    qs_mtm[1:] = qs[1:] * (h_mtm[:-1]/H)
    qs_mtm[0] = 0
    qs_sed[1:] = qs[1:] * (h_sed[:-1]/H)
    qs_sed[0] = 0
    
    # calculate change in each layer thickness
    h_ig[:-1] += ((-1/(porosity)) * ((np.diff(qs_ig)/dx) + (qs_ig[1:] * beta_ig))) * dt
    h_mtm[:-1] += ((-1/(porosity)) * ((np.diff(qs_mtm)/dx) + (qs_mtm[1:] * beta_mtm))) * dt
    h_sed[:-1] += ((-1/(porosity)) * ((np.diff(qs_sed)/dx) + (qs_sed[1:] * beta_sed))) * dt
    
    H_total = h_ig + h_mtm + h_sed
    dh = H - H_total
    
    h_ig += dh * theta_ig
    h_mtm += dh * theta_mtm
    h_sed += dh * theta_sed
    
    # set elevation equal to three sediment thicknesses
    # z[:-1] = h_ig[:-1] + h_mtm[:-1] + h_sed[:-1]
    
    # calc spatial change in percentage
#     percent_ig = h_ig/z
#     percent_mtm = h_mtm/z
#     percent_sed = h_sed/z

NameError: name 'qs_ig' is not defined

In [132]:
h_ig

array([1.0943984 , 0.89851109, 0.7918151 , 0.742652  , 0.72330047,
       0.71670303, 0.71472788, 0.71420213, 0.71407636, 0.71404906,
       0.71404365, 0.71404266, 0.71404249, 0.71404247, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404246,
       0.71404246, 0.71404246, 0.71404246, 0.71404246, 0.71404

In [133]:
h_mtm

array([0.15093787, 0.18358942, 0.20137443, 0.20956948, 0.21279524,
       0.21389499, 0.21422423, 0.21431187, 0.21433284, 0.21433739,
       0.21433829, 0.21433846, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433849,
       0.21433849, 0.21433849, 0.21433849, 0.21433849, 0.21433

In [134]:
h_sed

array([0.75466373, 0.91789949, 1.00681046, 1.04777852, 1.0639043 ,
       1.06940199, 1.07104789, 1.071486  , 1.0715908 , 1.07161355,
       1.07161806, 1.07161888, 1.07161902, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161905,
       1.07161905, 1.07161905, 1.07161905, 1.07161905, 1.07161

In [135]:
h_ig + h_mtm + h_sed

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])